In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
!pip -v install git+https://github.com/andreinechaev/nvcc4jupyter.git

Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m/UNKNOWN
Value for scheme.scripts does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/bin
sysconfig: /usr/bin
Value for scheme.data does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local
sysconfig: /usr
Additional context:
user = False
home

In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
%%cu 

#include <iostream> 

int main() { 
    //Here simple c program is only executed.
    //All complex CUDA program can be executed in the Google colab environment using this way.
    printf("CUDA is working\n");
    return 0; 
}

CUDA is working



In [17]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <random>

//Global variables to declare image dimensions

const int I_HEIGHT = 512;
const int I_WIDTH = 512;
const int M_H =3;
const int M_W =3;
const int arraySize = I_HEIGHT * I_WIDTH;
const int maskSize = M_H * M_W;
//Warp size
const int TILE = 32;


__global__ void conv(float* OUT, float* IN, float* M, int inw, int mw, int mh){
    // Get row and column to operate from thread coords
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    int bx = blockIdx.x;
    int by = blockIdx.y;

    int row = by*blockDim.y + ty;
    int col = bx*blockDim.x + tx;
    
    //calculate the padding radius of the kernel 
    int pw = (mw-1)/2; // for 3x3 kernel
    int ph = (mh-1)/2; 

    //mutex lock = synchronize threads
    __syncthreads();

    float val = 0.0f;
    for (int i = -ph; i <= ph; i++){
        for (int j = -pw; j<=pw; j++){
            val += IN[(row+ph - i) * inw + (col+pw-j)]*M[(i+ph)* mw + (j+pw)];
        }
    }
    // Save the result in the output matrix
    OUT[row*inw + col] = val; 
    __syncthreads();
}
cudaError_t convolution(float* out, float* in, float* m){
    float* dev_out = 0;
    float* dev_in = 0;
    float* dev_m = 0;

    cudaError_t cudaStatus; //debugging object
    dim3 Grid(I_WIDTH/TILE, I_HEIGHT/TILE,1);
    // dim3 Grid(16,16,1);
    dim3 Block(TILE,TILE,1);
    // dim3 Block(32,32,1);

    // Allocate the GPU memmory
    cudaMalloc((void**)&dev_out, I_HEIGHT*I_WIDTH*sizeof(float));
    cudaMalloc((void**)&dev_in, I_HEIGHT*I_WIDTH*sizeof(float));
    cudaMalloc((void**)&dev_m, M_H*M_W*sizeof(float));

    // get data from cpu
    cudaMemcpy(dev_in, in, I_WIDTH*I_HEIGHT*sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_m, m, M_W*M_H*sizeof(float), cudaMemcpyHostToDevice);

    conv<<<Grid,Block>>>(dev_out,dev_in,dev_m,I_WIDTH,I_HEIGHT,M_H,M_W);

    //check for any errors launching the kernel

    cudaStatus = cudaGetLastError();
    if(cudaStatus != cudaSuccess){
        fprintf(stderr, "conv2d kernel launch failed: %s\n", cudaGetErrorString(cudaStatus));
        // best practice to keep the memory garbarge free 
        goto Error;
    }
    cudaStatus = cudaDeviceSynchronize();
   if (cudaStatus != cudaSuccess)
    {
      fprintf(stderr, "cudaDeviceSynchronize returned error code %d after launching CONVOLUTION Kernel!\n", cudaStatus);
      goto Error;
    }

    // copy the results back to hosts

    cudaMemcpy(out, dev_out, I_HEIGHT*I_WIDTH*sizeof(float), cudaMemcpyDeviceToHost);
    Error:
      cudaFree(dev_out);
      cudaFree(dev_in);
      cudaFree(dev_m);

    return cudaStatus;

}
int main(){

  //dynamically allocate space for our iunput image array
  float* input_image = (float*)malloc(sizeof(float)*arraySize);
  //dynamically allocate space for our convolution result
  float* convolved_image = (float*)malloc(sizeof(float)*arraySize);

  float mask[maskSize] = {1/9.0f,1/9.0f,1/9.0f,1/9.0f,1/9.0f,1/9.0f,1/9.0f,1/9.0f,1/9.0f};

  for(int i = 0; i < arraySize; i++){
      input_image[i] = (rand()%256); // integers from 0 to 255
  }

  printf("input image");
  for(int i = 0; i<10; i++){
      for (int j = 0; j<10;j++)
      {
          printf("%.3f",input_image[i*I_WIDTH + j]);
      }
      printf("\n");
  }
  convolution(convolved_image,input_image,mask);
  return 0;
}

/tmp/tmpy9ceyc4v/fe84b39c-55e0-46dc-8d6c-372cf93ac6f7.cu(66): error: too many arguments in function call

1 error detected in the compilation of "/tmp/tmpy9ceyc4v/fe84b39c-55e0-46dc-8d6c-372cf93ac6f7.cu".

